**Prueba Técnica Data Engineer**

Presentado por: Buitrago Castillo Zuly

---
**Proyecto:** 
Sistema de consultas - ETL

---

In [ ]:
import pandas as pd
import numpy as np
import os,glob
import pandas_profiling
from datetime import datetime
import datetime as dt

### Carga de datos

Es necesario transformar el Json entregado para que la información cargue de manera correcta cuando se procede a realizar una carga en athena

Inicialmente se realiza una exploración sobre los datos cargados y las cantidades para un vez cargado se pueda validar si la carga fue exitosa

In [ ]:

import pandas as pd    
import seaborn as sns
file='technical_test_data_analyst.json'
data= pd.read_json(file, orient=str)
df = pd.DataFrame(data)
js=df.to_dict(orient='records')

In [ ]:

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1200 entries, 0 to 1199
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   activo         1200 non-null   int64  
 1   fecha_inicial  1200 non-null   int64  
 2   fecha_final    1200 non-null   int64  
 3   latitud        743 non-null    float64
 4   longitud       743 non-null    float64
 5   valorventa     573 non-null    float64
 6   valorarriendo  452 non-null    float64
 7   data           1200 non-null   object 
 8   id             1200 non-null   int64  
dtypes: float64(4), int64(4), object(1)
memory usage: 93.8+ KB


In [ ]:
data_inmueble=df.data.copy()

In [ ]:
import pandas as pd    
import seaborn as sns
file='technical_test_data_analyst.json'
data= pd.read_json(file)
type(data.data)

pandas.core.series.Series

## S3

###Libraries & keys

No dejamos las claves sobre el cuadernillo, si no estas se disponen en un archivo.txt que se ejecuta de manera local para la lectura de estas

In [ ]:
pip install boto3

In [ ]:
import boto3
from botocore.exceptions import NoCredentialsError
ACCESS_KEY=pd.read_csv('file_k.txt',sep=',',header=None)[0].tolist()[0]
SECRET_KEY=pd.read_csv('file_k.txt',sep=',',header=None)[1].tolist()[0]

### Create bucket
Se crear una funcion con el fin de poder suministrar el nombre del bucket que se desea crear

In [ ]:

def create_bucket(name_bucket, LocationConstraint):
    s3_client = boto3.client('s3', aws_access_key_id=ACCESS_KEY,
                  aws_secret_access_key=SECRET_KEY)
    try:
        s3_client.create_bucket(Bucket=name_bucket,CreateBucketConfiguration={'LocationConstraint':LocationConstraint})
        print("Bucket create Successful")
        return True
    except :
        print("Failed to create bucket")
        return False

create_bucket( 'technicaltestbucket', 'us-east-2')


Bucket create Successful


True

### Load file S3
Una vez creado el bucket y transformado el json se puede proceder a subir la informacion al s3

In [ ]:

def upload_to_aws(local_file, bucket, s3_file):
    s3 = boto3.client('s3', aws_access_key_id=ACCESS_KEY,
                      aws_secret_access_key=SECRET_KEY)
    try:
        s3.upload_file(local_file, bucket, s3_file)
        print("Upload Successful")
        return True
    except FileNotFoundError:
        print("The file was not found")
        return False
    except NoCredentialsError:
        print("Credentials not available")
        return False


uploaded = upload_to_aws('technical_test_data_analyst1.json', 'technicaltestbucket', 'technical_test_data_analyst.json')


### Uso Athena

### Creación de tabla 
1.Creación de Bd
2.indicar donde se va guardar los resultados de los querys ejecutados en configuraciones

```
CREATE EXTERNAL TABLE IF NOT EXISTS inmuebles.inmueble (
  activo STRING,
  fecha_inicial STRING,
  fecha_final STRING,
  latitud STRING,
  longitud STRING,
  valorventa STRING,
  valorarriendo STRING,
  data STRING,
  id STRING
)
ROW FORMAT SERDE 'org.openx.data.jsonserde.JsonSerDe'
WITH SERDEPROPERTIES (
 'serialization.format' = '1')
LOCATION 's3://technicaltestbucket/Data/'
```






### Creación de vista con atributos se emplean en la ejecución del analisis



```
CREATE OR REPLACE VIEW "View_data_analyze"
AS 
select id,
	activo,
	fecha_inicial,
	fecha_final,
	latitud,
	longitud,
	valorventa,
	valorarriendo,
case
				when ciudad like 'Bog%' then 'Bogota'
				when ciudad like 'Cart%' then 'Cartagena'
				when ciudad like 'Dos%' then 'Dosquebradas'
				when ciudad like 'Ita%' then 'Itagüí'
				when ciudad like 'Jamu%' then 'Jamundí' else ciudad
			end as ciudad,
			num_banos, antiguedad,area_lote
			From (

SELECT id,
	activo,
	fecha_inicial,
	fecha_final,
	latitud,
	longitud,
	CAST(valorventa as DECIMAL) AS valorventa,
	CAST(valorarriendo as DECIMAL) AS valorarriendo,
	CAST(coalesce(
		json_extract(replace(data, '''', '"'), '$.ciudad'),
		json_extract(replace(data, '''', '"'), '$.city')
	)AS VARCHAR(12)) AS ciudad,
	CAST(json_extract(replace(data, '''', '"'), '$.num_banos') AS INT) as num_banos,
	CAST(json_extract(replace(data, '''', '"'), '$.antiguedad') AS INT )as antiguedad,
	CAST(json_extract(replace(data, '''', '"'), '$.area_lote')AS int) as area_lote
FROM "inmuebles"."inmueble")t
```
Nota: se observa que en el atributo data la llave que contiene el nombre de la ciudad en unos registros esta como ciudad y en otros como city, se procede hacer la intregación de esta información o la normalización de estos datos




### Creación de consultas solicitadas

1. ¿Cómo están distribuidos los registros por ciudad?


```
CREATE OR REPLACE VIEW "View_registros_ciudad"
AS SELECT
    ciudad,
	count(*) as registros_ciudad
FROM View_data_analyze
WHERE ciudad is not null
GROUP BY ciudad
ORDER BY 2 DESC;
```

2.¿Cuál es el valor de venta y arriendo máximo y mínimo por ciudad?


```
CREATE OR REPLACE VIEW "view_value_Max_Min"
AS SELECT
    ciudad,
    MAX(valorventa) as max_venta,
    MIN(valorventa) as min_venta,
    MAX(valorarriendo) as max_arriendo,
    MIN(valorarriendo) as min_arriendo
	FROM View_data_analyze
	WHERE ciudad is not null AND valorventa!= 0 or valorarriendo !=0
	GROUP BY 1
	ORDER BY 1 ASC;
```

3.¿Listar el top 10 de los inmuebles más caros en valor de venta que presentan información de baños, antigüedad y área?


```
CREATE OR REPLACE VIEW "Inmuebles_mayor_Valor_e_informacion"
AS SELECT
    id,
    valorventa,
    num_banos,
    antiguedad,
    area_lote
	FROM View_data_analyze
WHERE num_banos !=0 and antiguedad  !=0
AND area_lote  !=0
ORDER BY valorventa DESC
LIMIT 10;
```








CREATE EXTERNAL TABLE IF NOT EXISTS inmuebles.inmueble (
  activo STRING,
  fecha_inicial STRING,
  fecha_final STRING,
  latitud STRING,
  longitud STRING,
  valorventa STRING,
  valorarriendo STRING,
  data STRING,
  id STRING
)
ROW FORMAT SERDE 'org.openx.data.jsonserde.JsonSerDe'
WITH SERDEPROPERTIES (
 'serialization.format' = '1')
LOCATION 's3://technicaltestbucket/Data/'


### Amazon SageMaker

Se emplea esta herramienta de aws para poder crear una instancia de jupyter notebook y desde este poder conectar directo con las consultas creadas en Athena y poder disponer de los datos para sus diferentes usos